In [1]:
import gurobipy as gp
from gurobipy import GRB
import time
import os
import pandas as pd

In [2]:
res_dir = 'results'
os.makedirs(res_dir, exist_ok=True)

output_dir = 'lp_files_gap'
os.makedirs(output_dir, exist_ok=True)

In [3]:
def add_cover_cuts(model):
    """
    Adds cover cuts to the given optimization model based on capacity constraints.

    Cover cuts are additional constraints that improve the linear relaxation 
    of integer programs by tightening the feasible region. This function focuses 
    on capacity constraints and applies cover inequalities to strengthen the model.

    Parameters:
    model (gurobipy.Model): The optimization model (Gurobi model) for which cover cuts
                            will be added.

    Behavior:
    1. The function first solves the given model to obtain an initial solution.
    2. It then identifies constraints related to capacity (identified by constraint names 
       starting with "capacity_"). These constraints are linear expressions.
    3. For each identified capacity constraint, a new binary variable `z_j` is created for 
       each variable `x_j` in the constraint. A secondary model (cover model) is built to 
       determine a minimal cover, which is a subset of variables that violate the capacity 
       constraint.
    4. The cover model is solved to find which variables should be part of the cover. 
       The cover inequality is added to the original model as a new constraint, ensuring 
       that the sum of selected variables (`x_j`) does not exceed the size of the cover minus one.
    5. After adding the cover cuts, the original model is re-optimized.

    Returns:
    gurobipy.Model: The updated optimization model with added cover cuts.
    """
    
    model.setParam('OutputFlag', 0)
    # Optimize the model to get an initial solution
    model.optimize()

    # Check if the model is solved optimally
    if model.status != GRB.OPTIMAL:
        print("Model did not solve to optimality.")
        return
    
    # Get the variable values from the current solution
    solution = {var.VarName: var.x for var in model.getVars()}

    # Iterate over all constraints in the model to find capacity constraints
    for constr in model.getConstrs():
        constr_name = constr.ConstrName
        
        # Identify capacity constraints (adjust the condition as per your constraints)
        if constr_name.startswith("capacity_"):
            # Get the linear expression of the constraint
            expr = model.getRow(constr)
            rhs = constr.RHS  # Right-hand side of the constraint
            
            # Create a new optimization model to find a cover
            cover_model = gp.Model(f"cover_{constr_name}")
            
            # Disable output from the cover model
            cover_model.setParam('OutputFlag', 0)
            
            # Create binary variables z_j for each x_j in the capacity constraint
            z_vars = []
            coeffs = []
            for i in range(expr.size()):
                var = expr.getVar(i)
                coeff = expr.getCoeff(i)
                coeffs.append((var, coeff))
                
                # Create a binary variable z_j corresponding to x_j
                z = cover_model.addVar(vtype=GRB.BINARY, name=f"z_{var.VarName}")
                z_vars.append(z)

            # Objective: maximize sum((1 - x_j) * z_j)
            cover_model.setObjective(
                gp.quicksum((1 - solution[var.VarName]) * z for z, (var, _) in zip(z_vars, coeffs)), GRB.MINIMIZE
            )
            
            # Constraint: sum(a_j * z_j) > b (cover constraint)
            epsilon = 1e-10  # A small positive number to replace the strict inequality
            cover_model.addConstr(gp.quicksum(coeff * z for z, (_, coeff) in zip(z_vars, coeffs)) >= rhs + epsilon, name="cover")

            # Optimize the cover model
            cover_model.optimize()
                
            # Check if the cover model found a solution
            if cover_model.status == GRB.OPTIMAL:
                # Collect the variables z_j that are set to 1 (indicating they are part of the cover)
                cover_vars = [var for z, (var, _) in zip(z_vars, coeffs) if z.x > 0.9]

                # Add the cover inequality as a new constraint in the original model
                if cover_vars:
                    cover_expr = gp.LinExpr()
                    for var in cover_vars:
                        cover_expr.add(var)  # Coefficient of 1 for each x_j corresponding to z_j in the cover

                    # Add the cover cut: sum of x_j <= |cover_vars| - 1
                    model.addConstr(cover_expr <= len(cover_vars) - 1, name=f"cover_cut_{constr_name}")
    
    # Re-optimize the original model with the added cover inequalities
    model.optimize()
    return model


In [6]:
def process_lp_files(lp_files_dir="lp_files_gap", time_limit=10):
    """
    Processes all LP files in the specified directory, optimizing each model
    until an integer solution is found or a time limit is reached.

    Args:
        lp_files_dir (str): Directory containing LP files.
        time_limit (int): Time limit for optimization in seconds.

    Returns:
        None
    """
    # Initialize a list to store the results
    results = []
    # Traverse the 'lp_files_gap' folder
    for root, dirs, files in os.walk(lp_files_dir):
        files = sorted(files)
        for file in files:
            if file.endswith(".lp"):  # Ensure we're working with .lp files
                # Full path of the input LP file
                input_file_path = os.path.join(root, file)
                model = gp.read(input_file_path)
                model.setParam('OutputFlag', 0)

                # Set time limit for the optimization process
                model.setParam('TimeLimit', time_limit)

                # Print or process the file name (for debugging or logging)
                print(f"Processing file: {file}")

                # Optimize the model initially to capture the starting objective value
                model.optimize()
                initial_objective = model.ObjVal

                # Store the objective value from the last iteration
                previous_objective = None
                loop_start_time = time.time()

                # Loop to solve the model until an integer solution is found or time limit is reached
                while True:
                    # Solve the model
                    model.optimize()

                    # Check if the solution is integer
                    is_integer = all(abs(v.x - round(v.x)) < 1e-10 for v in model.getVars())

                    # Check if the time limit has been reached
                    time_elapsed = time.time() - loop_start_time

                    # Break the loop if an integer solution is found or if the time limit has been reached
                    if is_integer or time_elapsed >= time_limit:
                        break

                    # Get the current objective value
                    current_objective = model.ObjVal

                    # If there was a previous objective, calculate the improvement
                    if previous_objective is not None:
                        improvement_percentage = ((previous_objective - current_objective) / previous_objective) * 100
                        # print(f"Improvement percentage: {improvement_percentage:.2f}%")

                    # Store the current objective value for the next iteration
                    previous_objective = current_objective

                    # Add cover cuts to the model
                    model = add_cover_cuts(model)

                # Capture the final objective value after the loop
                final_objective = model.ObjVal

                # print(f"Initial objective value: {initial_objective}")
                # print(f"Final objective value: {final_objective}")
                # print(f"Time elapsed: {time_elapsed:.2f} seconds")

                # # Print whether the solution is integer
                # if is_integer:
                #     print(f"Optimal integer solution found for file: {file}")
                # else:
                #     print(f"No optimal integer solution found for file: {file}")
                # print("")
                
                results.append({
                'file': file,
                'initial_value': initial_objective,
                'final_value': final_objective,
                'time_taken': time_elapsed
                })
    # Save the results to a CSV file   
    df = pd.DataFrame(results)
    df.to_csv(os.path.join(res_dir, 'knapsack_cuts.csv'), index=False)
    print(f"Results saved to {res_dir}/knapsack_cuts.csv")



In [7]:
if __name__ == "__main__":
    process_lp_files()

Read LP format model from file lp_files_gap/a05100.lp
Reading time = 0.00 seconds
obj: 105 rows, 500 columns, 1000 nonzeros
Processing file: a05100.lp
Initial objective value: 1697.7272727272727
Final objective value: 1697.8666666666668
Time elapsed: 20.01 seconds
No optimal integer solution found for file: a05100.lp

Read LP format model from file lp_files_gap/a05200.lp
Reading time = 0.00 seconds
obj: 205 rows, 1000 columns, 2000 nonzeros
Processing file: a05200.lp
Initial objective value: 3234.7391304347825
Final objective value: 3235.0
Time elapsed: 0.02 seconds
Optimal integer solution found for file: a05200.lp

Read LP format model from file lp_files_gap/a10100.lp
Reading time = 0.00 seconds
obj: 110 rows, 1000 columns, 2000 nonzeros
Processing file: a10100.lp
Initial objective value: 1358.556923076923
Final objective value: 1360.0
Time elapsed: 20.02 seconds
No optimal integer solution found for file: a10100.lp

Read LP format model from file lp_files_gap/a10200.lp
Reading time 